# Optimisation of the reservoir operating policy
In this Notebook we will see why and how to optimise the operating policy of a water reservoir system. An **operating policy** is a function that returns the operational decision to be made at any given time (such as the release volume for the next 24 hours) based on the conditions of the reservoir system (for instance, the reservoir storage, the demand forecast, the time of year, etc.) at that time. Differently from the release/pumping schedulings discussed in the previous Notebooks, which are optimised every time an operational decision must be made, the operating policy is optimised once and then applied forever after (or at least, until a revision of the policy is needed). In other words, the optimisation of the operating policy does not return a set of optimal operational decisions but rather a strategy for making optimal decisions ([Dobson et al, 2019](https://doi.org/10.1016/j.advwatres.2019.04.012)). It follows that, while the optimisation of release/pumping schedulings aims at maximising the benefits against the short- or mid-term forecasts of inflow and demand, the optimisation of the operating policy maximises the long-term benefits. These can be estimated by using (sufficiently long) historical time series or model projections.

<left><img src="../../util/images/Dam3.gif" width = "500px"><left>

Once again we consider a simple illustrative system where a reservoir is operated to supply water to a domestic consumption node, while ensuring a minimum environmental flow in the downstream river (also called “environmental compensation flow”) and maintaining the water level in the reservoir within prescribed limits. We use a mathematical model to link all the key variables that represent the reservoir dynamics (inflow, storage and outflows) and use model simulation/optimisation to determine the reservoir operating policy that optimizes the **long-term** (several years) system performance. We use the historical time series of inflows and water demand to estimate such long-term performance. The underpinning assumption here is that the system forcings observed over the past years are representative of the forcings that will drive the system in the future (if this assumption is not sensible, for instance because of ongoing changes that will likely impact the hydrological regime or demand pattern, then one may use model projections of inflow and demand in place of historical observations) 
<left> <img src="../../util/images/system_representation_IO1.png" width = "600px"><left>

We will use a simple form of operating policy where the reservoir release is only determined by the storage value, as in the Figure below. Higher storage values are associated to higher releases, which is useful for flood control purposes, whereas at low storage values less water is released to reduce the risk of future water shortages [(Loucks et al., 1981)](https://link.springer.com/book/10.1007/978-3-319-44234-1).
<left><img src="../../util/images/Policy_function.png" width = "400px"><left>

## Import libraries
To run this notebook we need to import some libraries:

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from platypus import NSGAII, Problem, Real, Integer # Import the optimizer

## Defining the shape of the operating policy
As anticipated, in this notebook we will use an operating policy that determines the reservoir release based on the storage value only. In particular, we will use a piece-wise linear function of the storage. The function is implemented in a series of submodules, that we can import with the following code:

In [2]:
from Modules.Interactive_release_policy import Interactive_policy_manual, Interactive_policy_auto
from irons.Functions.Data_management.Read_data import read_csv_data
from irons.Functions.Data_management.day2week2month import day2week
from irons.Functions.Reservoir_operating_policy.Operating_policy_functions import four_points_policy

In our code, the operating policy uses rescaled release and storage values. Specifically, the storage is scaled by the reservoir active capacity, so that in the operating policy function it varies between 0 (dead storage) and 1 (full storage). The release is scaled with respect to its mean, so that the operating policy returns the fraction (or multiple) of the mean release. 
The piece-wise linear function representing the operating policy is delineated by 4 points (x0, x1, x2 and x3): the minimimum and maximum (storage,release) points (x0 and x3 respectively), and two inflection points (x1, x2) where the slope of the function changes. The function returns a constant release (for instance, the target demand) when the storages stay between points x1 and x2; the release is reduced if the storage is below x1, or increased if it goes above x2. Let's now attribute a (tentative) value to the coordinates of these points, so that we can visualise the operating policy. For example:

In [3]:
# System constraints
env_min = 2 # ML/week - environmental compensation flow
u_mean  = 20 # ML/week - the (long-term) mean release   
u_min   = env_min # ML/week - the release at minimum storage 
u_max   = 60 # ML/week - the maximum release capacity
u_ref   = 20 # ML/week - the target demand 
u_0     = u_min/u_mean # (rescaling of u_min) 
u_1     = u_max/u_mean # (rescaling of u_max) 
u_ref   = u_ref/u_mean # (rescaling of u_ref) 
s_ref_1 = 0.2 # storage fraction at which 1st change in slope occurs 
s_ref_2 = 0.8 # storage fraction at which 2nd change in slope occurs
s_max   = 150 #  ML - maximum storage (=reservoir capacity)
s_min   = 0 # ML - minimum storage (set to zero for now)
### Operating policy defining points ###
x0 = [0,       u_0]
x1 = [s_ref_1, u_ref]
x2 = [s_ref_2, u_ref]
x3 = [1,       u_1]

param = [x0, x1, x2, x3, u_mean]

We can now create and plot the operating policy with the following code:

In [4]:
### Release fraction ###
u_frac = four_points_policy(param)/u_mean
### Storage fraction ###
s_frac = np.arange(0,1.01,0.01)
### Axis characteristics ###
x_sc_0 = LinearScale(min=0,max=1.03);y_sc_0 = LinearScale(min=0,max=4)
x_ax_0 = Axis(label='Storage fraction', scale=x_sc_0)
y_ax_0 = Axis(label='Release fraction', scale=y_sc_0, orientation='vertical')
### Plot ###
policy_function_points = Scatter(x = [x0[0], x1[0], x2[0], x3[0]], 
                            y = [x0[1], x1[1], x2[1], x3[1]],
                            colors=['red'],stroke = 'lightgray',
                            scales={'x': x_sc_0, 'y': y_sc_0},
                            names = ['x0','x1','x2','x3'])
policy_function_0 = Lines(x = s_frac, y = u_frac,
                       colors=['blue'],stroke = 'lightgray',
                       scales={'x': x_sc_0, 'y': y_sc_0})
### Figure characteristics ###
fig_0 = plt.Figure(marks = [policy_function_0,policy_function_points],title = 'Operating policy', axes=[x_ax_0, y_ax_0],
                    layout={'width': '500px', 'height': '450px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_0])

## Optimising the operating policy by trial and error (manual optimisation)
In this section we will refine the parameters of the operating policy (that is, in our example, the coordinates of points x0,x1,x2,x3) by trying to increase the system performance when simulated against the historical inflows, evaporation and water demand data. 

### Loading historical inflows, evaporation and water demand data
Let's assume we want to look at 100 weeks from 2014 to 2015, and load the evaporation, inflow and demand observations for this period from a file.

In [5]:
N = 100 # weeks
### Load evaporation data ###
inputs_folder_path = 'Inputs/'
clim_data_file = 'clim_data_2014_15.csv'
dates_day, clim_data_day  = read_csv_data(inputs_folder_path, clim_data_file)
dates, e_data_week, e_data_week_cum = day2week(N,dates_day,clim_data_day[:,0])
### Load inflow data ###
I_data_file = 'inflow_data_2014_15.csv'
dates_day, I_data_day = read_csv_data(inputs_folder_path, I_data_file)
dates, I_data_week, I_data_week_cum = day2week(N,dates_day,I_data_day[:,0])
### Load demand data ###
d_data_file = 'demand_data_2014_15.csv'
dates_day, d_data_day = read_csv_data(inputs_folder_path, d_data_file)
dates, d_data_week, d_data_week_cum = day2week(N,dates_day,d_data_day)

**Historical inflows**

In [6]:
I_data_file = 'inflow_data_2014_15.csv'
dates_day, I_data_day = read_csv_data(inputs_folder_path, I_data_file)
dates, I_data_week, I_data_week_cum = day2week(N,dates_day,I_data_day[:,0])

**Historical demand**

In [9]:
d_data_file = 'demand_data_2014_15.csv'
dates_day, d_data_day = read_csv_data(inputs_folder_path, d_data_file)
dates, d_data_week, d_data_week_cum = day2week(N,dates_day,d_data_day)

Plot the inflow time series:

In [10]:
# Axis characteristics
x_sc_1 = DateScale();y_sc_1 = LinearScale(min=0,max=40)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
inflow_plot = plt.bar(dates,I_data_week[:,0],colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
#Figure characteristics
fig_1a = plt.Figure(marks = [inflow_plot],title = 'Inflow', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1a])

Plot the demand time series:

In [7]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(dates,d_data_week[:,0],colors=['gray'],stroke = 'lightgray',opacities = [0.7]*N, 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1b])

### Definition of reservoir simulation function and initial storage
Next, we need to import the iRONs function that implements the reservoir simulation (this function iteratively applies the mass balance equation so to reconstruct the temporal evolution of the reservoir variables over the simulation period). We also need to specify the initial storage volume to start the simulation.

In [8]:
### Import the reservoir simulation function ###
from irons.Functions.Reservoir_system_simulation.Res_sys_sim import Res_sys_sim
### Define initial storage for simulation ###
s_0 = 80 # ML - initial storage volume

### Definition of the system objectives
Last, we need to define the objectives that we want to be maximised by the operating policy. As anticipated in the Introduction of this Notebook, our illustrative reservoir is operated to support domestic supply while maintaining the reservoir level above a prescribed target (this could be, for example, because the quality of the water deteriorates when levels are low, requiring more costly treatment).
We will pursue the first objective by minimising the following Total Squared Deficit (TDC) with respect to the historical water demand:

$$TSD = \sum_{t=1}^{N} [ \ max( \ 0, \ d(t)-u(t) \ ) \ ]^2 $$

where N is the length of the simulation period and d(t) is the water demand for each time-interval in that period. Notice that the function $max(0,...)$ enables us to only count the difference between demand d and release u when this is positive, that is, when the release u is smaller than the demand d, and a water shortage is indeed produced. Also, the squaring is a 'mathematical trick' to make sure that larger deficit amounts are given more weight than smaller ones. This translates the fact that small deficit amounts are easier to mitigate and hence more acceptable, while larger ones can cause disproportionately severe impacts and should be avoided as much as possible.

We are also interested in minimising the chances that the reservoir level goes below a minimum threshold. We measure how well this criterion is satisfied by the following Minimum Storage Violation (MSV) function:

$$MSV = \sum_{t=1}^{N} [ \ max ( \ rc - s(t) , \ 0) \ ] $$

where, again, N is the length of the simulation period, s is the reservoir storage, and rc is the minimum reservoir storage threshold that should preferably not be transpassed. For our case, let's set this threshold to 40 ML.

In [9]:
rc = np.array([40]*(N+1)) # (ML)  minimum reservoir storage threshold

### Determining the optimal operating policy via interactive visualisation

Now use the sliders to modify the parameters of the operating policy in a way that minimises the Total Squared Deficit (TSD) and the Minimum Storage Violation (MSV).

In [10]:
fig_1a,fig_1b,fig_1c, u_ref,s_ref_1,s_ref_2 = Interactive_policy_manual(N,
                                                                        I_data_week[1:,0], e_data_week[1:,0], 
                                                                        s_0, s_min, s_max, 
                                                                        u_0, u_1, u_mean, u_max, 
                                                                        env_min, d_data_week[1:,0], 
                                                                        rc)

Plot  *CAN WE REMOVE THIS BIT OF TEXT AND JUST HAVE ONE BOX OF CODE?*

In [11]:
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [widgets.VBox([u_ref,s_ref_1,s_ref_2],layout=Box_layout), fig_1a],layout=Box_layout),fig_1b,fig_1c],layout=Box_layout)

## From manual to automatic optimization 
As we have seen, when we deal with two conflicting objectives, we cannot find a solution that optimise both simoultaneously. If we prioritize one objective, the other one is deteriorated: there is a trade-off between the two. It would then be interesting to explore this tradeoff, and find a set of operating policies that produce different optimal combinations of the two objectives. However, this is too cumbersome to do manually, so we can use a multi-objective optimisation algorithm to do that for us. The algorithm will automatically test a very large number of combinations of the policy parameters u_ref, s_ref_1, s_ref_2, until it finds a set of combinations that realise (approximately) optimal tradeoffs.

Here we use a multi-objective optimisation algorithm called NSGAII, which is implemented in the Python Platypus package. For more information about these methods and tools, see [Deb et al, 2002](https://ieeexplore.ieee.org/document/996017) and the [Platypus webpage](https://platypus.readthedocs.io). The code to run the optimisation is the following:

In [12]:
def auto_optim(vars):

    u_ref   = vars[0]
    s_ref_1 = vars[1]
    s_ref_2 = vars[2]
    
    x0 = [0,       u_0]
    x1 = [s_ref_1, u_ref]
    x2 = [s_ref_2, u_ref]
    x3 = [1,       u_1]
    param = [x0, x1, x2, x3, u_mean] # Group parameters into a vector
    
    Qreg = {'releases' : {'file_name' : 'Reservoir_operating_policy.Operating_policy_functions',
                         'function' : 'four_points_policy',
                         'param': param},
            'inflows' : [],
            'rel_inf' : []}
    
    Qenv, Qspill, u, I_reg, s, E = Res_sys_sim(I_data_week[1:,0], e_data_week[1:,0], s_0, s_min, s_max, env_min, d_data_week[1:,0], Qreg)
    
    TSD = (np.sum((np.maximum(d_data_week[1:,0]-u,[0]*N))**2)).astype('int')
    MSV = (np.sum((np.maximum(rc-s,[0]*(N+1))))).astype('int')
    
    constraints = [s_ref_2-s_ref_1]
    
    return [TSD, MSV], constraints

problem = Problem(3,2,1)
Real0 = Real(0,u_1); Real1 = Real(0, 1); Real2 = Real(0, 1)

problem.types[:] = [Real0] + [Real1] + [Real2]
problem.constraints[:] = ">=0"
problem.function = auto_optim

population_size = 30
algorithm = NSGAII(problem,population_size)
algorithm.run(1000) # Number of iterations

results1_optim = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

sol_optim = [algorithm.result[i].variables for i in range(population_size)]

#### Plot the optimisation results
We can visualise the tradeoffs between the two objectives in one plot, called Pareto front, which displays the combination of the two objective values in correspondence to a set of optimised solutions. Click on one point in the Pareto front to visualise the operating policy that generates that performance, and associated storage time series.  What do you think would be a balanced solution?

In [13]:
fig_pf, fig_2a,fig_2b,fig_2c = Interactive_policy_auto(N, 
                                                       I_data_week[1:,0], e_data_week[1:,0], 
                                                       s_0, s_min, s_max, 
                                                       u_0, u_1, u_mean, u_max,
                                                       env_min, d_data_week[1:,0], 
                                                       rc, 
                                                       results1_optim,results2_optim,sol_optim)

Plot

In [14]:
Box_layout = widgets.Layout(justify_content='center')
widgets.VBox([widgets.HBox(
    [fig_pf, fig_2a],layout=Box_layout),fig_2b,fig_2c],layout=Box_layout)

### References 

Deb K. et al (2002) A fast and elitist multiobjective genetic algorithm: NSGA-II, IEEE Transactions on Evolutionary Computation, 6(2), 182-197, doi:10.1109/4235.996017.

Dobson B. et al (2019) An argument-driven classification and comparison of reservoir operation optimization methods, Advances in Water Resources, 128, 74-86.

Loucks D. P. et al (1981) Water resource systems planning and analysis, Prentice-Hall.

# Questionnaire: Section 2 of 2
Now that you are done with the Notebooks, could you please answer the questions of Section 2 of 2 of the questionnaire (click on Next button that you will find at the end of Section 1 of 2 of the questionnaire)?